In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

from models.mlp import BlackBoxModel

from models.rbf import RBFNet
from models.svm import LinearSVM
from explainers.model import Model
from utils.datasets import dataset_loader

pd.set_option('display.max_columns', None)

%reload_ext autoreload
%autoreload 2


In [ ]:
def bold(string):
    return "\033[1m" + string + "\033[0m"

In [ ]:
name = 'compas'
dropped_features = []#UCIDatasets().continuous_features[dataset]
dataset = dataset_loader(name, dropped_features=dropped_features, n_bins=None)

In [ ]:
dataset.data

In [ ]:
X_train, y_train, X_test, y_test, mean, std = dataset.get_split(normalise=False, shuffle=False,
                                                                     return_mean_std=True)
prop1s = round(np.average(y_train)*100, 2)
print(bold("Proportion of 1s in Training Data:") + " {}%".format(prop1s))

In [ ]:
X = pd.DataFrame(X_train)
X.columns = dataset.features[:-1]
X_train = pd.DataFrame(X_train)
X_train.columns = dataset.features[:-1]
X_test = pd.DataFrame(X_test)
X_test.columns = dataset.features[:-1]
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)
print(bold("Dataset:") + " {}\n".format(name.replace('_', ' ').title()))
X

In [ ]:
target_name = 'Status'

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

seed = 43
np.random.seed(seed)
torch.manual_seed(seed)

std = X_train.std()
mean = X_train.mean()

for col in ['Priors_Count', 'Time_Served']:
    X_train[col] = (X_train[col] - X_train[col].mean()) / X_train[col].std()
    X_test[col] = (X_test[col] - X_test[col].mean()) / X_test[col].std()

X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values


X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

model_raw = BlackBoxModel(input_dim=X_train.shape[1], hidden_dim=10).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model_raw.parameters(), lr=0.01)

model_raw.train()
for epoch in range(300):
    outputs = model_raw(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


model_raw.eval()
with torch.no_grad():
    logits = model_raw(X_test_tensor)
    preds = (torch.sigmoid(logits) > 0.5).float()
    accuracy = (preds == y_test_tensor).float().mean().item()
accuracy

model = Model(model=model_raw, backend="pytorch", data=None)


In [ ]:
sample_num = 50
X_test = pd.DataFrame(X_test, columns=X.columns)


# indice = pd.Index(np.random.choice(len(X_test), size=sample_num, replace=False))
indice = pd.Index([ 753,  582,  548,  113,  174,  420,  309,  998,  413, 1054, 1171,
             275,  771, 1135,  415,  478,  609,  168,  332,  933,  490,  906,
            1073,  128,  107, 1136,   43, 1168,  327,  610,  596,  631, 1205,
             534,  597,  220,  911,  198,  743,  985,  231,  865, 1088,  715,
             306,  101,  438,   78, 1223,   49,  936,  736,  243,  868, 1096,
             937,  381,  767,   63,  323,   44,  943, 1098,  482,  575,  254,
             529,  286,  992,  123,   76,  218, 1110,  545,  292,  701, 1226,
             844,  363,  811,  754,  266,  904,  958,  590, 1222,  233,   70,
             566,   23,  155,  707,   58,  428, 1008, 1057,  342,  629,   81,
            1127],
           dtype='int64')

df_explain = X_test.loc[indice]

# y_target = torch.distributions.beta.Beta(0.1, 0.9).sample((sample_num,))
y_test = pd.Series(y_test.reshape(-1))
y_true = y_test.loc[indice]

y = model(torch.FloatTensor(df_explain.values))

In [ ]:
def postprocessing(counterfactual_X):

    prior_count_col = counterfactual_X['Priors_Count']
    time_served_col = counterfactual_X['Time_Served']
    counterfactual_X = (counterfactual_X>0.5).replace({False:0 ,True:1})
    counterfactual_X['Priors_Count'] = prior_count_col
    counterfactual_X['Time_Served'] = time_served_col
    
    return counterfactual_X

## GLOBE_CE

In [ ]:
from explainers.globe_ce import GLOBE_CE

In [ ]:
normalise = None

# AReS initiated to determine bin widths for costs
from explainers.ares import AReS

X_for_ares = (
    dataset.data.drop(columns=[target_name])
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
    .astype(np.float32)
)

ares = AReS(model=model_raw, dataset=dataset, X=X_for_ares, n_bins=10, normalise=normalise)  # Use raw model for AReS
bin_widths = ares.bin_widths


In [ ]:
# example of ordinal features usage
ordinal_features = ['Present-Employment'] if name == 'german_credit' else []
# initialise GLOBE_CE
globe_ce = GLOBE_CE(model=model_raw, dataset=dataset, X=df_explain, affected_subgroup=None,
                    dropped_features=dropped_features, ordinal_features=ordinal_features, delta_init='zeros',
                    normalise=normalise, bin_widths=bin_widths, monotonicity=None, p=1)

In [ ]:
globe_ce.sample(n_sample=sample_num, magnitude=2, sparsity_power=1,  
                idxs=None, n_features=5, disable_tqdm=False,  
                plot=True, seed=0, scheme='random', dropped_features=dropped_features)
delta = globe_ce.best_delta  # pick best delta
globe_ce.select_n_deltas(n_div=3)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=globe_ce.deltas_div.shape[0], dpi=150)
fig.set_figwidth(12)
fig.set_figheight(3)
plt.subplots_adjust(wspace=0.3)
for i in range(globe_ce.deltas_div.shape[0]):
    delta_cost = globe_ce.deltas_div[i] * globe_ce.feature_costs_vector
    cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    j, k = 0, 0
    for feature in globe_ce.features_tree:
        if not globe_ce.features_tree[feature]:
            ax[i].bar(range(j, j+1), delta_cost[j], hatch='/',
                        linewidth=1, edgecolor='black', color=cycle[k%len(cycle)])
            j += 1
            k += 1
        else:
            feature_values = globe_ce.features_tree[feature]
            n_f = len(feature_values)
            ax[2].bar(range(j, j+n_f), delta_cost[j:j+n_f], color=cycle[k%len(cycle)])
            j += n_f
            k += 1
    ax[i].set_title(f'Direction {i+1}')
    ax[i].set_xlabel('Feature Index')
    ax[i].set_ylabel('Cost')
plt.show()

In [ ]:
n_div = globe_ce.deltas_div.shape[0]
min_costs = np.zeros((n_div, globe_ce.x_aff.shape[0]))
min_costs_idxs = np.zeros((n_div, globe_ce.x_aff.shape[0]))
for i in range(n_div):  
    cor_s, cos_s, k_s = globe_ce.scale(globe_ce.deltas_div[i], disable_tqdm=False, vector=True)  
    min_costs[i], min_costs_idxs[i] = globe_ce.min_scalar_costs(cos_s, return_idxs=True, inf=True)  

In [ ]:
ces = globe_ce.round_categorical(globe_ce.x_aff+globe_ce.best_delta) if globe_ce.n_categorical else globe_ce.x_aff+globe_ce.best_delta
counterfactual_X_global_ce = pd.DataFrame(ces, columns=X_test.columns)
counterfactual_X_global_ce = postprocessing(counterfactual_X_global_ce)
counterfactual_y_global_ce = model_raw.predict(counterfactual_X_global_ce.values)

In [ ]:
print('Coverage (Globe CE):', counterfactual_y_global_ce.sum()/len(counterfactual_y_global_ce))

In [ ]:
factual_X = pd.DataFrame(globe_ce.x_aff, columns=df_explain.columns)
factual_y = model_raw.predict(factual_X.values)

In [ ]:
y_target = torch.ones(factual_X.shape[0])

In [ ]:
costs_vector = globe_ce.feature_costs_vector

## AReS

In [ ]:
# AReS initiated to determine bin widths for costs
from explainers.ares import AReS
ares = AReS(model=model_raw, dataset=dataset, X=factual_X, n_bins=10, normalise=normalise)  # Use raw model for AReS  # 1MB

In [ ]:
ares.generate_itemsets(apriori_threshold=0.2, max_width=None, # defaults to e2-1
                       affected_subgroup=None, save_copy=False)
ares.generate_groundset(max_width=None, RL_reduction=True,
                        then_generation=None, save_copy=False)
lams = [1, 0]  # can play around with these lambda values
ares.evaluate_groundset(lams=lams, r=194, save_mode=1,
                        disable_tqdm=False, plot_accuracy=True)
ares.select_groundset(s=194)
ares.optimise_groundset(lams=lams, factor=1, print_updates=False,
                        print_terms=False)

In [ ]:
counterfactual_X_ares = pd.DataFrame(ares.R.cfx_matrix[0], columns=X_test.columns)
counterfactual_X_ares = postprocessing(counterfactual_X_ares)
counterfactual_y_ares = model_raw.predict(counterfactual_X_ares.values)

In [ ]:
print('Coverage (AReS):', counterfactual_y_ares.sum()/len(counterfactual_y_ares))

## Diverse Counterfactual Explanation (DiCE)

In [ ]:
backend = 'PYT'  

In [ ]:
import dice_ml

m = dice_ml.Model(model=model_raw, backend='PYT')  

factual_X_ext = factual_X.copy()
factual_X_ext[target_name] = factual_y

dice_features = factual_X.columns.drop(['Race = Asian', 'Race = Other']).to_list()

d = dice_ml.Data(dataframe=factual_X_ext, continuous_features=dice_features, outcome_name = target_name)

dice_explainer = dice_ml.Dice(d, m, method="random") 

In [ ]:
dice_results = dice_explainer.generate_counterfactuals(query_instances=factual_X, features_to_vary=dice_features, desired_class="opposite", total_CFs=1)

In [ ]:
dice_df_list = []
for cf in dice_results.cf_examples_list:
    cf_df = cf.final_cfs_df
    dice_df_list.append(cf_df)

counterfactual_X_dice = pd.concat(dice_df_list).reset_index(drop=True).drop(target_name, axis=1)
counterfactual_X_dice = postprocessing(counterfactual_X_dice)

In [ ]:
counterfactual_y_dice = model_raw.predict(counterfactual_X_dice.values)

In [ ]:
print('Coverage (DiCE):', counterfactual_y_dice.sum()/len(counterfactual_y_dice))

In [ ]:
print("="*80)
print("DICE: Computing OT Distance between CF predictions and target")
print("="*80)
y_target_tensor = y_target

dice_y_prob = model.predict_proba(counterfactual_X_dice.values)
dice_y_prob_tensor = torch.FloatTensor(dice_y_prob)


from explainers.distances import WassersteinDivergence
wd = WassersteinDivergence()
ot_dist_dice_y, _ = wd.distance(
    y_s=dice_y_prob_tensor,
    y_t=y_target_tensor,
    delta=0.1
)

print(f"DICE Y Probability OT Distance to Target: {ot_dist_dice_y:.6f}")

## Distributional Counterfactual Explanation (DCE)

In [ ]:
from explainers.dce import DistributionalCounterfactualExplainer

delta = 1e-5
alpha = 0.05
N = 10

explain_columns = df_explain.columns

explainer = DistributionalCounterfactualExplainer(
    model=model_raw, 
    df_X=factual_X, 
    explain_columns=explain_columns,
    y_target=y_target, 
    lr=0.1, 
    n_proj=N,
    delta=delta,
    costs_vector=None)

In [ ]:
explainer.optimize(U_1=0.01, U_2=0.2, l=0.7, r=0.85, max_iter=100, tau=1e3)

In [ ]:
counterfactual_X_dce = pd.DataFrame(explainer.best_X.detach().numpy(), columns=df_explain.columns)
counterfactual_X_dce = postprocessing(counterfactual_X_dce)

dtype_dict = dataset.data.drop(columns=[target_name]).dtypes.apply(lambda x: x.name).to_dict()

for k, v in dtype_dict.items():
    if k in counterfactual_X_dce.columns:
        if v[:3] == 'int':
            counterfactual_X_dce[k] = counterfactual_X_dce[k].round().astype(v)
        else:
            counterfactual_X_dce[k] = counterfactual_X_dce[k].astype(v)

counterfactual_y_prob_dce = pd.DataFrame(explainer.y.detach().numpy(),columns=[target_name], index=counterfactual_X_dce.index)
counterfactual_y_dce = np.int64((counterfactual_y_prob_dce.values > 0.5).reshape(-1))

In [ ]:
print('Coverage (DCE):', counterfactual_y_dce.sum()/len(counterfactual_y_dce))

## DISCOVER (Distributional Counterfactual Optimization with Variance Reduction)

DISCOVER is our improved version of DCE that uses advanced sampling strategies for better optimization.

In [ ]:
from explainers.DCESolver import DCESolver
from explainers.cone_sampling.monte_carlo import MonteCarloStrategy
# from explainers.data import DataLoader
from data_loader.compas import CompasData
from explainers.model import Model
discover_seed = seed
print(f"DISCOVER seed: {discover_seed}")

In [ ]:
# Create a minimal DataLoader wrapper for DISCOVER
class BaselineDataWrapper:
    def __init__(self, df_factual, df_explain, y_test, continuous_cols, categorical_cols):
        self.df = df_factual.copy()
        self.X_train = df_explain.values
        self.y_train = y_test

        self.explain_columns = df_explain.columns.tolist()
        self.continuous_columns = continuous_cols
        self.categorical_columns = categorical_cols

        self.mean = df_factual[continuous_cols].mean().to_dict()
        self.std = df_factual[continuous_cols].std().to_dict()

        for col in continuous_cols:
            if self.std[col] == 0:
                self.std[col] = 1.0
        
    def get_X_init(self):
        import torch
        return torch.from_numpy(self.X_train).float()
    
    def get_y_target(self):
        import torch
        return torch.ones(len(self.X_train))


continuous_features_discover = ['Priors_Count', 'Time_Served']
categorical_features_discover = [col for col in df_explain.columns if col not in continuous_features_discover]

discover_data = BaselineDataWrapper(
    df_factual=factual_X,
    df_explain=factual_X,  
    y_test=factual_y,  
    continuous_cols=continuous_features_discover,
    categorical_cols=categorical_features_discover
)

print(f"DISCOVER data wrapper created: {len(factual_X)} samples")
print(f"Continuous features: {len(continuous_features_discover)}")
print(f"Categorical features: {len(categorical_features_discover)}")

In [ ]:
# Initialize DISCOVER solver
discover_solver = DCESolver(model=model, data=discover_data)

# Initialize MonteCarlo sampling strategy
discover_strategy = MonteCarloStrategy(
    explainer=discover_solver,
    random_state=discover_seed,
    cone_angle=3.14159/4,  # π/4
    use_cone_sampling_categorical=True,
    use_cone_sampling_continuous=True,
    categorical_step=1.2,
    continuous_step=0.1,
    temperature=2.0,
    h = 2
)

print("DISCOVER solver and strategy initialized")


In [ ]:
# Run DISCOVER optimization
counterfactual_X_discover_df = discover_solver.explain(
    df_factual=factual_X,
    explain_columns=factual_X.columns.tolist(),
    categorical_columns=categorical_features_discover,
    continuous_columns=continuous_features_discover,
    y_target=torch.ones(len(factual_X)),
    strategy=discover_strategy,
    X_init=False,  
    n_proj=10,
    delta=1e-5,
    costs_vector=None,
    U_1=0.8,  
    U_2=0.6,  
    alpha=0.05,
    l=0.2,    
    r=1,    
    kappa=0.05,
    max_iter=45,
    num_trials=50,
    bootstrap=True,
    callback=False,
    top_k=1,
    save_results=False,  
    use_global_ranges=False,
    target_ot_y=ot_dist_dice_y
)

print(f"\nDISCOVER optimization completed")
print(f"Best Q found: {discover_solver.best_Q:.6f}")
print(f"Best iteration: {discover_solver.best_iter}")

In [ ]:
counterfactual_X_discover = postprocessing(counterfactual_X_discover_df)
counterfactual_y_discover = model_raw.predict(counterfactual_X_discover.values)
print(f"DISCOVER counterfactuals: {len(counterfactual_X_discover)} samples")
print(f"Coverage (DISCOVER): {counterfactual_y_discover.sum()/len(counterfactual_y_discover):.4f}")

## Distance Evaluation

In [ ]:
from explainers.distances import SlicedWassersteinDivergence, WassersteinDivergence
from scipy.stats import gaussian_kde, entropy
from numpy.linalg import LinAlgError

def compute_distance(X_s, X_t):
    if isinstance(X_s, pd.DataFrame):
        X_s = X_s.apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.float32)
        X_s = torch.FloatTensor(X_s.values)
    if isinstance(X_t, pd.DataFrame):
        X_t = X_t.apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.float32)
        X_t = torch.FloatTensor(X_t.values)

    if isinstance(X_s, np.ndarray):
        X_s = torch.FloatTensor(X_s.astype(np.float32))
    if isinstance(X_t, np.ndarray):
        X_t = torch.FloatTensor(X_t.astype(np.float32))

    if X_s.ndim == 1:
        wd = WassersteinDivergence()
        distance, _ = wd.distance(X_s, X_t, delta=0.1)
    else:
        swd = SlicedWassersteinDivergence(
                dim=X_s.shape[1], n_proj=5000
        )
        distance, _ = swd.distance(X_s, X_t, delta=0.1)
    return distance.item()


def compute_kl_divergence(X_s, X_t):
    kl_divergences = []
    for i in range(X_s.shape[1]):  # Iterate over columns (features)
        try:
            # Estimate probability distributions using KDE
            kde_s = gaussian_kde(X_s[:, i])
            kde_t = gaussian_kde(X_t[:, i])

            # Evaluate the densities on a linear space of the same range
            x_min = min(X_s[:, i].min(), X_t[:, i].min())
            x_max = max(X_s[:, i].max(), X_t[:, i].max())
            x = np.linspace(x_min, x_max, 1000)

            # Compute the KL divergence (entropy)
            kl_div = entropy(kde_s(x), kde_t(x))
        except LinAlgError:
            # Catch the singular matrix error and set the divergence to infinity
            kl_div = np.inf

        kl_divergences.append(kl_div)

    # Aggregate the KL divergences
    total_kl_divergence = np.sum(kl_divergences)  # Or use np.mean for average
    return total_kl_divergence

def gaussian_kernel(x, y, sigma=1.0):
    """Compute the Gaussian kernel between x and y"""
    return np.exp(-np.linalg.norm(x - y) ** 2 / (2 * sigma ** 2))

def mmd(X_s, X_t, kernel=gaussian_kernel):
    """Compute the Maximum Mean Discrepancy (MMD) between two samples X_s and X_t"""
    n = X_s.shape[0]
    m = X_t.shape[0]

    # Calculate the kernel values between all points in the first sample
    XX = np.sum([kernel(X_s[i], X_s[j]) for i in range(n) for j in range(n)])
    
    # Calculate the kernel values between all points in the second sample
    YY = np.sum([kernel(X_t[i], X_t[j]) for i in range(m) for j in range(m)])
    
    # Calculate the kernel values between all points across the two samples
    XY = np.sum([kernel(X_s[i], X_t[j]) for i in range(n) for j in range(m)])

    return XX / (n ** 2) + YY / (m ** 2) - 2 * XY / (n * m)


In [ ]:
cov_ares = counterfactual_y_ares.sum()/len(counterfactual_y_ares)  
cov_global_ce = counterfactual_y_global_ce.sum()/len(counterfactual_y_global_ce)
cov_dice = counterfactual_y_dice.sum()/len(counterfactual_y_dice)
cov_dce = counterfactual_y_dce.sum()/len(counterfactual_y_dce) 
cov_discover = counterfactual_y_discover.sum()/len(counterfactual_y_discover)

In [ ]:
print('Coverage (AReS):', cov_ares)  
print('Coverage (Globe CE):', cov_global_ce)
print('Coverage (DiCE):', cov_dice)
print('Coverage (DCE):', cov_dce)  
print('Coverage (DISCOVER):', cov_discover) 

In [ ]:
ot_dist_ares = compute_distance(X_s=counterfactual_X_ares, X_t=factual_X)  
ot_dist_global_ce = compute_distance(X_s=counterfactual_X_global_ce, X_t=factual_X)
ot_dist_dce = compute_distance(X_s=counterfactual_X_dce, X_t=factual_X) 
ot_dist_dice = compute_distance(X_s=counterfactual_X_dice.dropna(), X_t=factual_X)
ot_dist_discover = compute_distance(X_s=counterfactual_X_discover, X_t=factual_X) 

print('X Distance (AReS):', ot_dist_ares)  
print('X Distance (Globe CE):', ot_dist_global_ce)
print('X Distance (DiCE):', ot_dist_dice)
print('X Distance (DCE):', ot_dist_dce)  
print('X Distance (DISCOVER):', ot_dist_discover)

In [ ]:
# ========== Y Distance Evaluation (Predicted Y Probability vs Target Y) ==========
print("="*80)
print("Y Distance Evaluation: Predicted Y Probability vs Target Y")
print("="*80)

y_target_tensor = y_target

# Get predicted y PROBABILITIES (not binary 0/1) for each method's counterfactual results
y_prob_globe = model_raw.predict_proba(counterfactual_X_global_ce.values)[:, 1]
y_prob_ares = model_raw.predict_proba(counterfactual_X_ares.values)[:, 1]
y_prob_dice = model_raw.predict_proba(counterfactual_X_dice.values)[:, 1]
counterfactual_X_dce = counterfactual_X_dce.apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.float32)
y_prob_dce = model_raw.predict_proba(counterfactual_X_dce.values)[:, 1]
y_prob_discover = model_raw.predict_proba(counterfactual_X_discover.values)[:, 1]

# Convert to tensors
y_prob_globe_tensor = torch.FloatTensor(y_prob_globe)
y_prob_ares_tensor = torch.FloatTensor(y_prob_ares)
y_prob_dice_tensor = torch.FloatTensor(y_prob_dice)
y_prob_dce_tensor = torch.FloatTensor(y_prob_dce)
y_prob_discover_tensor = torch.FloatTensor(y_prob_discover)

# Compute OT distance between predicted probabilities and target y (all 1s)
ot_dist_y_globe = compute_distance(X_s=y_prob_globe_tensor, X_t=y_target_tensor)
ot_dist_y_ares = compute_distance(X_s=y_prob_ares_tensor, X_t=y_target_tensor)
ot_dist_y_dice = compute_distance(X_s=y_prob_dice_tensor, X_t=y_target_tensor)
ot_dist_y_dce = compute_distance(X_s=y_prob_dce_tensor, X_t=y_target_tensor)
ot_dist_y_discover = compute_distance(X_s=y_prob_discover_tensor, X_t=y_target_tensor)

print(f'Y Probability Distance (AReS vs Target): {ot_dist_y_ares:.6f}')
print(f'Y Probability Distance (GLOBE vs Target): {ot_dist_y_globe:.6f}')
print(f'Y Probability Distance (DiCE vs Target): {ot_dist_y_dice:.6f}')
print(f'Y Probability Distance (DCE vs Target): {ot_dist_y_dce:.6f}')
print(f'Y Probability Distance (DISCOVER vs Target): {ot_dist_y_discover:.6f}')



In [ ]:
# ========== Y (Risk) CDF Curve Visualization with Diagnostics ==========
import matplotlib.pyplot as plt
import numpy as np

y_factual = model_raw.predict_proba(factual_X.values)[:, 1]
y_ares_cf = model_raw.predict_proba(counterfactual_X_ares.values)[:, 1]
y_globe_cf = model_raw.predict_proba(counterfactual_X_global_ce.values)[:, 1]
y_dice_cf = model_raw.predict_proba(counterfactual_X_dice.values)[:, 1]
y_discover_cf = model_raw.predict_proba(counterfactual_X_discover.values)[:, 1]
y_target_vals = y_target
y_dce_cf = model_raw.predict_proba(counterfactual_X_dce.values)[:, 1]

fig, ax = plt.subplots(figsize=(10, 7), dpi=120)
ax.plot(np.sort(y_dce_cf), np.linspace(0, 1, len(y_dce_cf)),
        label="DCE", color="#F39C12",
        linewidth=2, alpha=0.85)

ax.plot(np.sort(y_factual), np.linspace(0, 1, len(y_factual)),
        label="Factual", color="gray",
        linewidth=2.5, alpha=0.7, linestyle=":")

ax.plot(np.sort(y_ares_cf), np.linspace(0, 1, len(y_ares_cf)),
        label="AReS", color="#FF6B6B",
        linewidth=2, alpha=0.85)

ax.plot(np.sort(y_globe_cf), np.linspace(0, 1, len(y_globe_cf)),
        label="GLOBE", color="#7B2CBF",
        linewidth=2, alpha=0.85)

ax.plot(np.sort(y_dice_cf), np.linspace(0, 1, len(y_dice_cf)),
        label=f"DiCE", color="#4ECDC4",
        linewidth=2, alpha=0.85)

ax.plot(np.sort(y_discover_cf), np.linspace(0, 1, len(y_discover_cf)),
        label="DISCOVER", color="#1E88E5",
        linewidth=2.5, alpha=0.9)

ax.plot(np.sort(y_target_vals), np.linspace(0, 1, len(y_target_vals)),
        label="Target", color="black",
        linestyle="--", linewidth=3, alpha=0.95)

ax.axvline(0.5, color="red", linestyle=":", linewidth=1.5, alpha=0.4)

ax.set_xlabel("Risk Probability (Y)", fontsize=18, fontweight="bold")
ax.set_ylabel("Cumulative Probability (Quantile)", fontsize=18, fontweight="bold")
ax.set_title("Empirical CDF Comparison (Y)",
            fontsize=25, fontweight="bold", pad=15)

ax.legend(loc="upper left", fontsize=14, frameon=True, shadow=True)
ax.grid(True, alpha=0.3, linestyle="--", linewidth=0.5)
ax.set_xlim([0, 1.05])
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig("cdf.pdf", format="pdf", bbox_inches="tight")
plt.show()



In [ ]:
def plot_five_methods_comparison(dice_dist, globe_dist, discover_dist, ares_dist, dce_dist, top_n=10, save_path=None):

    plt.rcParams.update({
        'font.size': 30,         
        'axes.titlesize': 30,
        'axes.labelsize': 30,
        'xtick.labelsize': 15,
        'ytick.labelsize': 15,
        'legend.fontsize': 30
    })


    dice_sorted = np.sort(dice_dist)[::-1]
    globe_sorted = np.sort(globe_dist)[::-1]
    discover_sorted = np.sort(discover_dist)[::-1]
    ares_sorted = np.sort(ares_dist)[::-1]
    dce_sorted = np.sort(dce_dist)[::-1]

    n_samples = len(dice_sorted)
    x_positions = np.arange(n_samples)

    global_max = max(dice_sorted.max(), globe_sorted.max(), discover_sorted.max(),
                     ares_sorted.max(), dce_sorted.max())
    y_limit = global_max * 1.15

    # fig, axes = plt.subplots(5, 1, figsize=(16, 18), sharex=True, dpi=120)
    fig, axes = plt.subplots(5, 1, figsize=(16, 15), sharex=True, dpi=120)

    methods = [
        ('ARES', ares_sorted, '#2ECC71'),
        ('GLobe', globe_sorted, '#3498DB'),
        ('DiCE', dice_sorted, '#E74C3C'),
        ('DCE', dce_sorted, '#F39C12'),    
        ('DISCOVER', discover_sorted, '#9B59B6'),

    ]

    for idx, (name, sorted_dist, color) in enumerate(methods):
        ax = axes[idx]

        bars = ax.bar(x_positions, sorted_dist,
                      width=1.0,
                      alpha=0.7,
                      color=color,
                      edgecolor='none')

        for i in range(min(top_n, n_samples)):
            bars[i].set_color(color)
            bars[i].set_alpha(0.95)
            bars[i].set_edgecolor('black')
            bars[i].set_linewidth(0.5)

        ax.set_ylim([0, y_limit])
        ax.set_ylabel('OT Distance', fontsize=25, fontweight='bold')
        ax.set_title(f'{name}', fontsize=25, fontweight='bold', loc='left', pad=10)
        ax.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.7)

        textstr = (f'Mean: {sorted_dist.mean():.4f}\n'
                   f'Std:  {sorted_dist.std():.4f}\n'
                   f'Max:  {sorted_dist.max():.4f}')
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.85)
        # ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=10,
        #         verticalalignment='top', horizontalalignment='right',
        #         bbox=props, family='monospace')

    axes[4].set_xlabel('Sample Index',
                       fontsize=25, fontweight='bold')
    axes[4].set_xlim([-1, n_samples])

    fig.suptitle(f'Samplewise OT ({n_samples} samples)',
                 fontsize=40, fontweight='bold', y=0.995)

    # plt.tight_layout()
    plt.tight_layout(pad=0.4)
    fig.subplots_adjust(hspace=0.3)

    if save_path is not None:
        fig.savefig(save_path, format='pdf', bbox_inches='tight')

    plt.show()


factual_X_aligned = factual_X.reset_index(drop=True).copy()
counterfactual_X_dice_aligned = counterfactual_X_dice.reset_index(drop=True).copy()
counterfactual_X_globe_aligned = counterfactual_X_global_ce.reset_index(drop=True).copy()
counterfactual_X_discover_aligned = counterfactual_X_discover.reset_index(drop=True).copy()

counterfactual_X_ares_aligned = counterfactual_X_ares.reset_index(drop=True).copy()
counterfactual_X_dce_aligned = counterfactual_X_dce.reset_index(drop=True).copy()

print('ARES individual OT distances...')
ares_individual_distances = compute_ot_individual_distances(
    factual_X_aligned, counterfactual_X_ares_aligned
)


print('GLobe individual OT distances...')
globe_individual_distances = compute_ot_individual_distances(
    factual_X_aligned, counterfactual_X_globe_aligned
)

print('DiCE individual OT distances...')
dice_individual_distances = compute_ot_individual_distances(
    factual_X_aligned, counterfactual_X_dice_aligned
)

print('DCE individual OT distances...')
dce_individual_distances = compute_ot_individual_distances(
    factual_X_aligned, counterfactual_X_dce_aligned
)

print('DISCOVER individual OT distances...')
discover_individual_distances = compute_ot_individual_distances(
    factual_X_aligned, counterfactual_X_discover_aligned
)

plot_five_methods_comparison(
    dice_individual_distances,
    globe_individual_distances,
    discover_individual_distances,
    ares_individual_distances,
    dce_individual_distances,
    top_n=10,
    save_path='ot_comparison_compas_mlp.pdf'
)


In [ ]:
print('X MMD (AReS):', mmd(X_s=counterfactual_X_ares.values, X_t=factual_X.values)) 
print('X MMD (Globe CE):', mmd(X_s=counterfactual_X_global_ce.values, X_t=factual_X.values))
print('X MMD (DiCE):', mmd(X_s=counterfactual_X_dice.dropna().values, X_t=factual_X.values))
print('X MMD (DCE):', mmd(X_s=counterfactual_X_dce.values, X_t=factual_X.values))  
print('X MMD (DISCOVER):', mmd(X_s=counterfactual_X_discover.values, X_t=factual_X.values)) 

In [ ]:
print('X KL-Divergence (AReS):', 
      compute_kl_divergence(X_s=counterfactual_X_ares.values, X_t=factual_X.values))
print('X KL-Divergence (Globe CE):', 
      compute_kl_divergence(X_s=counterfactual_X_global_ce.values, X_t=factual_X.values))
print('X KL-Divergence (DiCE):', 
      compute_kl_divergence(X_s=counterfactual_X_dice.dropna().values, X_t=factual_X.values))
print('X KL-Divergence (DCE):', 
      compute_kl_divergence(X_s=counterfactual_X_dce.values, X_t=factual_X.values))
print('X KL-Divergence (DISCOVER):',
      compute_kl_divergence(X_s=counterfactual_X_discover.values, X_t=factual_X.values))

In [ ]:
ares_diff_pct = []  
globe_ce_diff_pct = []
dice_diff_pct = []
dce_diff_pct = [] 
discover_diff_pct = []
for column in df_explain.columns:
    ares_pct = (counterfactual_X_ares[column] - factual_X[column]).abs().sum() / (1e-7 + factual_X[column].abs().sum()) 
    globe_ce_pct = (counterfactual_X_global_ce[column] - factual_X[column]).abs().sum() / (1e-7 + factual_X[column].abs().sum())
    dice_pct = (counterfactual_X_dice[column] - factual_X[column]).abs().sum() / (1e-7 + factual_X[column].abs().sum())
    dce_pct = (counterfactual_X_dce[column] - factual_X[column]).abs().sum() / (1e-7 + factual_X[column].abs().sum())  
    discover_pct = (counterfactual_X_discover[column] - factual_X[column]).abs().sum() / (1e-7 + factual_X[column].abs().sum())

    ares_diff_pct.append({column: ares_pct}) 
    globe_ce_diff_pct.append({column: globe_ce_pct})
    dice_diff_pct.append({column: dice_pct})
    dce_diff_pct.append({column: dce_pct}) 
    discover_diff_pct.append({column: discover_pct})

## Cost Evaluation

In [ ]:
def compute_cost(delta, costs_vector):
    return np.linalg.norm(delta @ np.diag(costs_vector)) 


def compute_absolute_difference(counterfactual_X, factual_X):
    columns = counterfactual_X.columns.drop(['Priors_Count', 'Time_Served'])
    diff_list = []

    for column in columns:
        diff_list.append((counterfactual_X[column] - factual_X[column]).abs().mean())

    return np.nanmean(diff_list)

def compute_statistic_difference(counterfactual_X, factual_X, metric, columns):
    diff_list = []
    for column in columns:
        val_cf = counterfactual_X[column].agg(metric)
        val_f = factual_X[column].agg(metric)
        diff_list.append(abs(val_cf - val_f)/(abs(val_f)) * 100)

    return np.nanmean(diff_list)


In [ ]:
ares_delta = (counterfactual_X_ares - factual_X).dropna().values 
globe_ce_delta = (counterfactual_X_global_ce - factual_X).values
dce_delta = (counterfactual_X_dce - factual_X).dropna().values  
dice_delta = (counterfactual_X_dice - factual_X).dropna().values
discover_delta = (counterfactual_X_discover - factual_X).dropna().values

In [ ]:
print('Cost (AReS):', compute_cost(ares_delta, costs_vector)) 
print('Cost (Globe CE):', compute_cost(globe_ce_delta, costs_vector))
print('Cost (DiCE):', compute_cost(dice_delta, costs_vector))
print('Cost (DCE):', compute_cost(dce_delta, costs_vector))  
print('Cost (DISCOVER):', compute_cost(discover_delta, costs_vector))

## Diversity

In [ ]:
def compute_average_pairwise_distance(counterfactual_X):
    n = len(counterfactual_X)
    total_distance = 0
    count = 0

    for i in range(n):
        for j in range(i+1, n):
            dist = np.linalg.norm(counterfactual_X.iloc[i] - counterfactual_X.iloc[j])
            total_distance += dist
            count += 1

    if count > 0:
        average_distance = total_distance / count
    else:
        average_distance = 0

    return average_distance


In [ ]:
diversity_factual = compute_average_pairwise_distance(factual_X)

In [ ]:
print('Diversity (Factual)', diversity_factual)

In [ ]:
diversity_ares = compute_average_pairwise_distance(counterfactual_X_ares)  
diversity_global_ce = compute_average_pairwise_distance(counterfactual_X_global_ce)
diversity_dice = compute_average_pairwise_distance(counterfactual_X_dice.dropna())
diversity_dce = compute_average_pairwise_distance(counterfactual_X_dce)  
diversity_discover = compute_average_pairwise_distance(counterfactual_X_discover)

In [ ]:
print('Diversity (AReS):', diversity_ares)  
print('Diversity (Globe CE):', diversity_global_ce)
print('Diversity (DiCE):', diversity_dice)
print('Diversity (DCE):', diversity_dce)  
print('Diversity (DISCOVER):', diversity_discover)

In [ ]:
print('Effective Diversity (AReS):', diversity_ares/ot_dist_ares * cov_ares)  
print('Effective Diversity (Globe CE):', diversity_global_ce/ot_dist_global_ce * cov_global_ce)
print('Effective Diversity (DiCE):', diversity_dice/ot_dist_dice * cov_dice)
print('Effective Diversity (DCE):', diversity_dce/ot_dist_dce * cov_dce)  
print('Effective Diversity (DISCOVER):', diversity_discover/ot_dist_discover * cov_discover)